In [1]:
from PIL import Image, ImageOps, ImageFont
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets
from ipywidgets import interactive, HBox, VBox
from ascii_art_conversion_simple import img2ascii_arr
from ascii_art_conversion_v2 import img2ascii_arr_v2
from img_processing import preprocess_img, DITHER_MODES
from braille_art import img2braille_arr, get_braille_chars
from generate_ascii_palette import *

In [2]:
ASCII_SET = get_asciis()
BRAILLE_SET = get_braille_chars()
ASCII_AND_BRAILLE_SET = ASCII_SET + BRAILLE_SET

In [3]:
FONT = ImageFont.truetype("fonts/CascadiaMono.ttf", 16)
IMG_PATH = "imgs/irad_grad.bmp"

In [4]:
orig_img = Image.open(IMG_PATH).convert("L")
#orig_img = ImageOps.invert(orig_img)

print(orig_img.size)

(512, 512)


In [5]:
def i_preprocess_img(scale_factor, contrast, brightness, quantize_colors, eq, dither):
    proc_img = preprocess_img(orig_img, 0.1, contrast, brightness, eq, quantize_colors, dither)
    plt.imshow(proc_img, cmap='gray', vmin=0, vmax=255, interpolation='none')

    plt.figure(figsize=(9, 3))
    plt.xticks([x for x in range(0, len(proc_img.histogram()), 25)])
    plt.bar([x for x in range(0, len(proc_img.histogram()))], proc_img.histogram())
    plt.grid()
    plt.show()

    return proc_img

interactive_preprocess = interactive(i_preprocess_img,
            scale_factor=(0.01, 1, 0.01),
            contrast=(0, 2, 0.01),
            brightness=(0, 2, 0.01),
            eq=(0, 1, 0.01),
            quantize_colors=(1, 256, 1),
            dither=DITHER_MODES)

controls = VBox(interactive_preprocess.children[:-1])
display(HBox((controls, interactive_preprocess.children[-1])))

In [6]:
proc_img = interactive_preprocess.result

In [7]:
palette, palette_br = generate_1_1_palette(ASCII_SET, FONT, 12, normalize=True)
ascii_arr = img2ascii_arr(proc_img, palette, palette_br, dither=DITHER_MODES.FS)
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j], sep='', end='')
    print('\n', end='')

   ....,,','~,;~;;^;~;";"!"^"~~!^~^;~`~,,:'.`...   
 ..`.,::`^-~!~-^"""""=""|""/""<""""!;^~;^^`^,`:.:..
.,_:`;!!^--^"""<"|"><c\|=>?v+v"v>"?"""""^-;;,,:'`..
.,:,~`~;~"""?"<|=>>+?L?*\?\?vLucc<*vv>"""""!;~!_'',
:'-_;~!""">"=>cc?=iulz1nnYCuxnvT>jc<+|>v"<"";^^^-,'
';^~^"""*"=<*+|Tnsunjjl]lItxyCzTj1Luv><?v">""";^;_'
,--^""L"v|=Llrx1xJJoZ3a[S77feyXf]xfxls\<>+L">"""-!!
-~!"""|<>|}=Yjns]7IbV6pKG9Ew443p[3alYCC}+>?+"v""!^~
~!""<"c=|\}1Ci7{yd2bPEpq4gKdEG6Pp{[IXV(}n++++"?"""!
^;""+<\?vxYThfX4U9E6PMRgM&#%8&mmHp5X{V11(nu+\L"<"^!
-""+"<<|nC)nayaPqH&R&gRQWWN$Q&%D6mPGdoee}i*LvLc""""
^""/"/*(cJ)othp]E6gg8QWW@$@BBWQ&gDHq22FZ}xxY<c"="""
-""+>>>>u(ja2yApKRR%0NN@@@Q@WBNDRH5Hd7Ffz)i*/=\">"-
^"""\/*+nCll7XPm650D0QBQQ@$@Q$gR&&Eba5[X1n(=\c*""""
;"<"*"?(*Ci]hIVHE9p0%8$NWNQ$MW&8EE52w2Xril=xc+"*"""
;"""\cv<?11uuh{ZEK66DD##R&0R#RE&b4d]3VUu((=vL/"""";
!~""""cL>cTJxUyUU2wmK6p&KR%55H5pyU{htos)x|(<"="=""-
!;;""|">L+c1liiey3e2hqZ4GAP5ppV5ZHeFnCrT>?=<">"";~!
`-~"""|"//\cLz}ju1]Zf]oFa{X[y3tFUT(jr)*vL*"/""""-;`
'-;!-""""+"<

In [8]:
palette, palette_br = generate_1_2_palette(ASCII_SET, FONT, (8,8), normalize=True)
ascii_arr = img2ascii_arr_v2(proc_img, palette, palette_br, dither=DITHER_MODES.FS)
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j], sep='', end='')
    print('\n', end='')

   . -::-:::~;-~~~!~!~!!!!!!!!!~~~!!;-!:---:- -    
  : :::--;-~!!~~*!*<+/>*||+|+\/<<<!!!~!!!;-::-: :  
 ---::-!;~!~!+<>>+**+\|=|>>|=\+=*>*|<<<~~~~;--::- -
::-:-~~!!!+<|>+=|*\c)C1l}YhCU)>h=|\|>>>/<!~!!;---::
-:--~!~~><|++\=ICV(tU]{J]2}[I]t2}YhY\|\**|<~~!~;---
-:~!~!=<>>>*+}Y}tUU22{kZV%hb]V{}IhIU(h\|=\==/!!~;--
:~!!!>~*+\=h)IhU}{GO6%db6G%d%654Ah2hI1t((>+\\!|~!~:
;~!///|>Y(JhhtIZ%Z%d%bGGGQbd%46%66ZK]11}2Y\>>|<~~!~
;~!+~\*+tYUhhUb46%6GW$WBWW$WRWRGG44dESh1t]l\+*+<<~!
!~</=+C>VV1IK4%b%bW#QWWWWWWWWWW#W$d%43S]{IVC+>>|~!!
~~>=\|Y2Y[2H[6%G#&DWWWW@W@@W@WWWWR%d%6HK{V2(*Y|>/<~
!<~=+*CcCJX54G4NdWWWWW@@@@@@@@WWWWR#%G6KJ2JJ)*+|!+~
~/<*|Y|tVI]bKGbNWNWWW@@@@@@@@@@WWWN86bGHOV}I>l>*\~!
!~*|=\(2h{UEd4b#6WWWW@@@@@@@@@WWWWRNbG%SI1}IY==></~
!<!=|+CiCJ2S43dNRWRWWWW@@@@@WWWWWB$$G%5XIVUC(===<~~
!!//+>+])}[[E6%GR4W0WWWWWWWWWWW8W80463EO}}I)l>+|<!!
~~<~|=\))[IhAVG4Gd0GW0WDWWBW#W$8d4b4O3XJUJl+>*\<!/!
!~!<<+|*C>1[1tA2%bdB68dN8%W#R%DbDd33S2]][CY+Y=/~/!~
-~!!/!**(=l2C]1OI5A43%d#%NGbG%%X4X5]2{JYY=Y\>/~/!!-
:-~!<<//>*>C

In [9]:
ascii_arr = img2braille_arr(proc_img, dither=DITHER_MODES.FS)
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j], sep='', end='')
    print('\n', end='')

⠀⠀⠠⢀⠂⠔⡐⡐⢔⢐⢅⠢⡊⢔⡐⢅⠢⡂⡂⡂⡂⠄⠂⡀⠀
⠀⠌⡐⠄⢅⠕⡐⡅⡕⡌⡆⡇⡣⡣⡪⢪⢨⢢⠱⡐⢌⠌⡐⠠⢁
⠈⠔⡨⠨⡢⡑⡅⡇⣕⢕⡕⡵⡱⣕⢵⡱⡕⡕⡕⠥⡑⡌⢌⠌⢄
⢈⠢⡊⢜⠔⡕⡕⡵⡕⡧⣫⣞⣝⢮⣺⢜⡮⡪⡎⡇⡇⡕⢅⠪⡐
⢐⢅⠪⡊⡎⡮⡺⡪⡯⣞⣗⡷⣽⣽⣺⣳⢽⣹⣪⢺⡸⡸⡨⡊⠔
⠰⡐⢕⢕⢕⢵⢝⡽⣝⡾⣞⣿⢷⣿⣞⣷⡻⡮⡮⡣⡇⡇⡎⡜⡨
⠨⠪⡸⢸⢸⢪⡳⣝⡾⣽⣻⡿⣿⣿⣽⡾⣯⣻⡪⣏⢞⢜⢜⠔⢌
⠨⡱⡘⡜⡜⣕⢝⡞⣞⣽⣳⢿⣻⣽⢾⢯⣗⢗⣝⢮⢪⢣⢱⢑⠅
⠨⢂⢪⠸⡸⣘⢮⢺⢕⢷⢽⢽⢽⡺⡯⣳⡳⣝⢮⢺⢸⢸⢐⢅⠊
⢈⠢⠡⡱⡘⡔⡕⡵⡹⡱⣝⢮⡳⣝⢝⢮⢺⡸⡸⡸⡘⢔⠅⡢⠡
⢀⠊⠌⠔⢌⠢⡣⢱⢱⢹⢸⢸⢸⢸⢱⢱⢱⢱⢑⢅⠣⡑⠌⠄⠅
⠀⠄⠡⠑⡐⠡⡊⢌⠆⢕⢜⠸⡘⢜⠸⡘⢌⠢⡑⠔⡡⠂⠅⠌⠠
